In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import skimage
import time

In [ ]:
# remove some points with decay
# add progressive parameteres based on likelihood
# add "speeding up" in case of mouse movement or keyboard

In [ ]:
img = cv2.imread('../print.jpg')/255.0
max_size = 1000
H, W = img.shape[0], img.shape[1]
if H > W:
    if H > max_size:
        img = cv2.resize(img, (int(W * max_size / H)), max_size)
else:
    if W > max_size:
        img = cv2.resize(img, (max_size, int(H * max_size / W)))
        
H, W = img.shape[0], img.shape[1]
padding = int(W * 0.1 if H > W else H * 0.1)
zeros = np.zeros((H + padding * 2, W + padding * 2, 3))
zeros[padding: padding + H, padding: padding + W] = img
img = zeros
mask_img = np.zeros(img.shape)

blurs = dict()
for i in range(1, 100):
    blurs[i] = cv2.blur(img, (i, i))

In [ ]:
plt.imshow(img[:,:,::-1])

In [ ]:
class Stroke():
    def __init__(self, image, dispersion, amount):
        image_size = image.shape[:2]
        max_size = max(image_size)
        self.dispersion = dispersion
        x = np.random.randint(0, image_size[1], 1)
        y = np.random.randint(0, image_size[0], 1)
        self.center = np.concatenate([x, y], axis=0)
        
        self.particles_angle = np.random.uniform(0, 2 * np.pi, amount)
        self.particles_radius = np.fabs(np.random.normal(0, dispersion*2, amount))
        
        
#         self.positions = np.concatenate([x, y], axis=0)
#         self.positions = self.positions.reshape(1, 2).repeat(amount, axis=0)
#         self.positions = self.positions + np.random.randn(amount, 2) * (dispersion*2)
        self.angle = np.random.uniform(0, 2 * np.pi)
        self.rotation = np.random.uniform(-np.pi/30, np.pi/30)
        self.color = image[y[0], x[0]]
        self.speed = np.random.normal(max_size/200, 5, 1)
        self.status = 'running'
        self.ttl = np.random.normal(1)
        
        
    def get_mean_color(self, image, x, y):
        if 0 < x < image.shape[1] and 0 < y < image.shape[0]:
            return blurs[self.dispersion][int(y), int(x)]
        else:
            return self.color
        
        
    def update(self, image):  
        # add removal of some points
        self.last_center = self.center.copy()
        self.center[0] += np.cos(self.angle) * self.speed
        self.center[1] += np.sin(self.angle) * self.speed
#         self.particles_angle += self.rotation
        self.angle += self.rotation
        mean_color = self.get_mean_color(image, self.center[0], self.center[1])
        self.color = self.color - (self.color - mean_color)/2
        self.ttl -= np.random.uniform(0, 0.05)
        if self.ttl <= 0:
            self.status = 'deleted'
        
        
    def draw(self, image):
        offset_x = np.cos(self.particles_angle) * self.particles_radius
        offset_y = np.sin(self.particles_angle) * self.particles_radius
        positions = (self.center.reshape(2, 1) + [offset_x, offset_y]).T.reshape(-1, 1, 2)
        last_positions = (self.last_center.reshape(2, 1) + [offset_x, offset_y]).T.reshape(-1, 1, 2)
        points = np.concatenate([last_positions, positions], axis=1).astype(np.int32)
        cv2.polylines(image, points, False, self.color, 2)   
        self.particles_angle += self.rotation


In [ ]:
max_strokes = 10
strokes = [Stroke(img, np.random.randint(1, 8), np.random.randint(1, 20)) for _ in range(max_strokes)]

while True:
    strokes = [st for st in strokes if st.status != 'deleted']
    for _ in range(max_strokes - len(strokes)):
        strokes.append(Stroke(img, np.random.randint(1, 8), np.random.randint(1, 20)))
    prev = time.time()
    for st in strokes:
        st.update(img)
        st.draw(mask_img)
#     print('update and paint: ',(time.time() - prev)*1000)
        
    prev = time.time()
    cv2.imshow('BlindArtist', mask_img)
#     print('show ',(time.time() - prev)*1000)
    key = cv2.waitKey(1)
    if key == 27:      # esc
        break
        
cv2.destroyAllWindows() 